# Data Wrangling With SQL - Project Report


### Author - Manasi Sheth

<hr style="height:1px;color:black">

## Table of Contents

<li><a href="#intro">Introduction and Overview</a></li>
<li><a href="#wrangling">Problems Encountered in the Map</a></li>
<li><a href="#countcode">Tag Count Code</a></li>
<li><a href="#bad">Checking contents for bad characters</a></li>
<li><a href="#users">Exploring users</a></li>
<li><a href="#street">Overabbreviated Street Names</a></li>
<li><a href="#zipcodes">Postal Codes</a></li>
<li><a href="#phone">Telephone Numbers</a></li>
<li><a href="#database">Preparing for SQL Database</a></li>
<li><a href="#eda">Creating Database and Data Exploration</a></li>
<li><a href="#idea">Additional Ideas</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<hr style="height:1px;color:black">

<a id='intro'></a>
## Introduction and Overview
<hr style="height:1px;color:black">

For the project, Openstreet map of Des Moines, Iowa was downloaded from mapzen. The size of the uncompressed file is 187 MB. The sample file - "sample.osm", is attached with the project which is generated by taking every 10-th top level element.

For the data wangling project, First, the expected number of each tag that are present in osm file were analyzed, data was checked for bad characters, and list of the unique users that contributed to the file provided was generated. The data was then audited to find incorrect street types, zipcodes, phone numbers and then that data in incorrect format was cleaned programmatically to have standardized values. 

After the data cleaning was completed, the data was prepared to be imported into the database using SQL. The data was then explored and analyzed using SQL queries. 

<hr style="height:1px;color:black">

<a id='wrangling'></a>
## Problems Encountered in the Map

><a id='countcode'></a>
>### Tag Count Code
For this section, the expected number of each tag that are present in the osm file were analyzed. Open street map uses tags like nodes, ways and relations to present data for map. Ways correspond to street, or paths through a city. A specific point on the earth's surface defined by its latitude and longitude corresponds to a node. Relations are used to document the relationships between two or more data elements like nodes, ways etc. A tag usually consists of a key and a value and describe the meaning of particular element they are attached to. The code to count the number of each tags is as follows - 


In [36]:
import requests
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import csv
import codecs
import cerberus
import schema
import os

In [37]:
# Assign a variable for the osm file
des_moines_info = "des_moines.osm"

In [38]:
# This function count the number of each tag in the data for des moines osm file
#This function can be found in map_parser.py

def count_tags(filename):
        # YOUR CODE HERE
        store_tags = {}
        for _, val in ET.iterparse(filename):
            if val.tag in store_tags:
                store_tags[val.tag] += 1
            else:
                store_tags[val.tag] = 1
        return store_tags  

pprint.pprint(count_tags(des_moines_info))

{'bounds': 1,
 'member': 8707,
 'meta': 1,
 'nd': 1073070,
 'node': 915603,
 'note': 1,
 'osm': 1,
 'relation': 810,
 'tag': 335790,
 'way': 64668}


The above method count_tags, sets a new key for each unique tag type with a value of 1. When the tag is encountered again, the value is incremented by 1. From the output, we can see that the number of nodes = 915603, number of ways = 64668. Once the data is loaded into database, the number of records in the nodes and ways tables should tally with these numbers. 

><a id='bad'></a>
>### Checking contents for bad characters

In this section, the data is checked for bad characters using regular expressions. The code for it as follows - 

In [50]:
### This function checks data for bad characters. 
### This function can be found in map_parser.py

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        # YOUR CODE HERE
        
        keycontents = element.attrib['k']
        if(re.match(lower, keycontents) != None):
            keys["lower"] = keys["lower"] + 1
        elif(re.match(lower_colon, keycontents) != None):
            keys["lower_colon"] = keys["lower_colon"] + 1
        elif(re.match(problemchars, keycontents) != None):
            keys["problemchars"] = keys["problemchars"] + 1
        else:
            #print(keycontents)
            keys["other"] = keys["other"] + 1
        
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

pprint.pprint(process_map(des_moines_info))

{'lower': 151730, 'lower_colon': 130681, 'other': 53379, 'problemchars': 0}


The above function key_type, gives a count of each of the four tag categories in dictionary. The key "problemchars" gives count of tags with problematic characters, "lower" gives count of tags with valid and lowercase characters, "lower_colon" gives count of valid tags with a colon in their name, "other" contains tags that are not in the other three categories. 

><a id='users'></a>
>### Exploring users

The code below gives list of the unique users that contributed to the Openstreet map file.

In [51]:
## The code in this cell will give a list of the unique users that contributed to the file provided
## This function can be found in map_parser.py

def get_user(element):
    return element.attrib["uid"]


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if 'uid' in element.attrib:
            users.add(get_user(element))

    return users

pprint.pprint(len(process_map(des_moines_info)))

469


The number of unique users present for des moines osm file is 469. The above function, process_map iterates through the parsed file and then with the help of the function get_user returns set of unique userIDs.  

><a id='street'></a>
>### Overabbreviated Street Names

The next task involved auditing of the Openstreet map to find out the consistency and uniformity of the data. One of the problems which was observed was inconsistent street names. The street names were over-abbreviated in OSM data. For example - the street names were abbreviated as "Grand Ave", "Gateway Dr", "S West St" etc. To make sure the street names are consistent, the unexpected street types were fixed to the appropriate ones like "Ave" was replaced with "Avenue", "Blvd" was replaced by boulevard etc. The code to do is as follows - 

In [52]:
#### The code below is used to audit, clean and update the street names
#### The code below till update_name function is found in audit_update_street_type.ipynb
OSMFILE = "des_moines.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Junior"]


In [53]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

mapping = { "St": "Street",
            "St.": "Street",
            "ST":"Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Rd.,": "Road",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Cir": "Circle",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Hwy":"Highway",
            "Ct": "Court",
            "Pkwy": "Parkway",
            "Ln": "Lane",
            "Sq": "Square",
            "Ste": "Suite",
            "Real":"Real",
            "Saratoga":"Saratoga",
            "Alameda": "Alameda",
            "Way": "Way",
            "Expressway": "Expressway",
            "Jr":"Junior"
            }


In [54]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r", encoding="utf8")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types



{'117': {'North State Highway 117'},
 '150': {'Westown Pky Ste 150'},
 '28': {'Highway 28'},
 '5': {'Highway 5'},
 '6A': {'SW 9th St #6A'},
 'Ave': {'Grand Ave', '34th Ave', 'Carpenter Ave'},
 'Chase': {'Crescent Chase'},
 'Creek': {'Pring Creek'},
 'Dr': {'NW Urbandale Dr', 'Gateway Dr', 'Adventureland Dr'},
 'Guthrie': {'Guthrie'},
 'Northwest': {'Bass Pro Drive Northwest'},
 'Pkwy': {'Martin Luther King Jr Pkwy'},
 'Plaza': {'40th Plaza',
           '42nd Plaza',
           'Concord Plaza',
           'Lexington Plaza',
           'Woodland Plaza'},
 'Rd': {'Hickman Rd', 'Ashworth Rd'},
 'ST': {'SW 9TH ST'},
 'South': {'West 125th Street South'},
 'Southeast': {'Paine Street Southeast'},
 'Southwest': {'36th Avenue Southwest',
               '7th Avenue Southwest',
               '8th Street Southwest',
               'Carter Street Southwest'},
 'St': {'S 52nd St', 'S West St', 'SW 9th St', 'SE 14th St'},
 'University': {'University'},
 'Way': {'Keosauqua Way', 'Keosaqua Way', 'Wat

In the code, "Expected" is a predefined list of street names that we want after cleaning the street names. 

"mapping" is a dictionary which has mapping of what wrong street names would be present in the des moines osm file. The update_function which cleans the street names uses mapping.

The audit function opens up the des moines Openstreet map file to be parsed. Then the function iterates through the file. If there is a node or way tag, then the "tag" tags in them are examined to see if they have street names. If the street names are present, the tag of the street name is send to audit_street_type function.
The is_street_name function is used to find out if there is an address in the tag.
The audit_Street_type function searches through the "street_type_re" regular expression to see if it finds a match. If there is a match, it finds what term the string matched on. Else if there is no match, the tag is added to the dictionary of street type where key is the street type and value is the whole street name. 

Following are the audited street names - 

In [55]:
st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))


{'117': {'North State Highway 117'},
 '150': {'Westown Pky Ste 150'},
 '28': {'Highway 28'},
 '5': {'Highway 5'},
 '6A': {'SW 9th St #6A'},
 'Ave': {'Grand Ave', '34th Ave', 'Carpenter Ave'},
 'Chase': {'Crescent Chase'},
 'Creek': {'Pring Creek'},
 'Dr': {'NW Urbandale Dr', 'Gateway Dr', 'Adventureland Dr'},
 'Guthrie': {'Guthrie'},
 'Northwest': {'Bass Pro Drive Northwest'},
 'Pkwy': {'Martin Luther King Jr Pkwy'},
 'Plaza': {'40th Plaza',
           '42nd Plaza',
           'Concord Plaza',
           'Lexington Plaza',
           'Woodland Plaza'},
 'Rd': {'Hickman Rd', 'Ashworth Rd'},
 'ST': {'SW 9TH ST'},
 'South': {'West 125th Street South'},
 'Southeast': {'Paine Street Southeast'},
 'Southwest': {'36th Avenue Southwest',
               '7th Avenue Southwest',
               '8th Street Southwest',
               'Carter Street Southwest'},
 'St': {'S 52nd St', 'S West St', 'SW 9th St', 'SE 14th St'},
 'University': {'University'},
 'Way': {'Keosauqua Way', 'Keosaqua Way', 'Wat

After the auditing is done, the update_name function updates the street names by cleaning it. The code for update_function is as follows - 

In [56]:
def update_name(name, mapping):
    
    m = street_type_re.search(name)
    new_name = name
    if m:
        street_type = m.group()
        if street_type in mapping:
            new_name = street_type_re.sub(mapping[street_type], name)  
    return new_name


The update_name function searches if the street name is present in the regular expression "street_type_re" then it substitutes that old abbreviated street names with the expected cleaned name. The output of the function is as follows -  

In [57]:
#### display the output
for st_type, ways in st_types.items(): # changed this method from .iteritems() from 2.7 to 3.6's .items()  
        for name in ways:
            better_name = update_name(name, mapping)
            print (name, "=>", better_name)

Highway 5 => Highway 5
8th Street Southwest => 8th Street Southwest
7th Avenue Southwest => 7th Avenue Southwest
36th Avenue Southwest => 36th Avenue Southwest
Carter Street Southwest => Carter Street Southwest
NW Urbandale Dr => NW Urbandale Drive
Gateway Dr => Gateway Drive
Adventureland Dr => Adventureland Drive
University => University
Martin Luther King Jr Pkwy => Martin Luther King Jr Parkway
Grand Ave => Grand Avenue
34th Ave => 34th Avenue
Carpenter Ave => Carpenter Avenue
Hickman Rd => Hickman Road
Ashworth Rd => Ashworth Road
S 52nd St => S 52nd Street
S West St => S West Street
SW 9th St => SW 9th Street
SE 14th St => SE 14th Street
SW 9th St #6A => SW 9th St #6A
Paine Street Southeast => Paine Street Southeast
Westown Pky Ste 150 => Westown Pky Ste 150
Bass Pro Drive Northwest => Bass Pro Drive Northwest
Guthrie => Guthrie
Keosauqua Way => Keosauqua Way
Keosaqua Way => Keosaqua Way
Watson Powell Jr Way => Watson Powell Jr Way
Wolf Way => Wolf Way
Pring Creek => Pring Creek


From the above output, we can see that last word in the street name which was abbreviated, was cleaned to match its corresponding clean form from the list of expected street names.

><a id='zipcodes'></a>
>### Postal Codes

After auditing and cleaning the street names, the next task is to audit and clean the postal codes to maintain the uniformity. The Des Moines Openstreet Map data has postal codes in two formats like 50237 and 50313-4155. The idea is to update the postal codes to just keep 5 digits to maintain uniformity. Following is the code to audit and update the postal code - 

In [46]:
## The following code is used to audit the zipcodes and improve them 
## The code can be found in audit_update_zipcodes.ipynb
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = 'des_moines.osm'
postal_code_re = re.compile(r'^[0-9]{5}?$')

def audit_zipcodes(osmfile):
    osm_file = open(osmfile, "r", encoding="utf8")
    postal_codes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)): # TODO: Learn about what events param does
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_codes(tag):
                    m = postal_code_re.search(tag.attrib['v'])
                    if m:
                        postal_codes['formatted'].add(m.group())
                    else:
                        postal_codes['not_formatted'].add(tag.attrib['v'])
                
    osm_file.close()
    return postal_codes


def is_postal_codes(elem):
    return (elem.attrib['k'] == "addr:postcode")


The method is_postal_code is used to find postal code in the tag. 

The audit_zipcodes function opens up the des moines Openstreet map file to be parsed. Then the function iterates through the file. If there is a node or way tag, then the "tag" tags in them are examined to see if they have postal codes. Then the  function searches through the postal_code_re regular expression to see if it finds a match for postal code with 5 digits. If there is a match, it adds it in a dictionary of postal codes for key "formatted" and if the postal code is present which have more than 5 digits then it adds it in the dictionary of postal codes for key "unformatted". 

In [47]:
st_types = audit_zipcodes(OSMFILE)
pprint.pprint(dict(st_types))

{'formatted': {'50003',
               '50009',
               '50021',
               '50023',
               '50035',
               '50047',
               '50054',
               '50061',
               '50063',
               '50111',
               '50118',
               '50131',
               '50169',
               '50211',
               '50228',
               '50237',
               '50261',
               '50263',
               '50265',
               '50266',
               '50309',
               '50310',
               '50311',
               '50312',
               '50313',
               '50314',
               '50315',
               '50316',
               '50317',
               '50319',
               '50320',
               '50321',
               '50322',
               '50323',
               '50324',
               '50325',
               '50327'},
 'not_formatted': {'50021-4021',
                   '50021-9335',
                   '50021-9353',
            

After the auditing is done, the unformatted postal codes are updated to retain only 5 digits. The code to update the postal codes is as follows -

In [48]:
import zipcodes
def update_zipcode(zipcode):
    #return re.sub('(\d{5})-\d{4}', '\\1', zipcode)
    #print(zipcode)
    b = re.findall(r'\d+', zipcode)
    if b: 
        return b[0]

The update_zipcode function, with the help of regular expression searches if there are any zipcodes present in the string and if zipcode is present, the function returns the first 5 digits of the zipcode. The output of the function is as follows - 

In [49]:
st_types = audit_zipcodes(OSMFILE)

for st_type, ways in st_types.items(): # changed this method from .iteritems() from 2.7 to 3.6's .items()  
        for name in ways:
            better_name = update_zipcode(name)
            print (name, "=>", better_name)

50322 => 50322
50312 => 50312
50310 => 50310
50111 => 50111
50047 => 50047
50324 => 50324
50311 => 50311
50325 => 50325
50021 => 50021
50323 => 50323
50265 => 50265
50003 => 50003
50266 => 50266
50228 => 50228
50309 => 50309
50118 => 50118
50317 => 50317
50023 => 50023
50237 => 50237
50035 => 50035
50061 => 50061
50211 => 50211
50321 => 50321
50314 => 50314
50054 => 50054
50009 => 50009
50169 => 50169
50261 => 50261
50320 => 50320
50131 => 50131
50327 => 50327
50263 => 50263
50316 => 50316
50315 => 50315
50063 => 50063
50313 => 50313
50319 => 50319
50313-3431 => 50313
50312-2099 => 50312
50313-4627 => 50313
50313-4056 => 50313
50265-2053 => 50265
50023-3054 => 50023
50313-1206 => 50313
50265-5318 => 50265
50313-4175 => 50313
50023-7235 => 50023
50313-2213 => 50313
50021-9353 => 50021
50021-4021 => 50021
50311-4207 => 50311
50313-2712 => 50313
50021-9335 => 50021
50023-9723 => 50023
50313-1202 => 50313
50313-4155 => 50313
50313-2798 => 50313
50131-1824 => 50131
50313-4237 => 50313


><a id='phone'></a>
>### Telephone Numbers

The next problem which was present in Des Moines Openstreet map file was lack of uniformity in phone numbers. The phone numbers are in different formats like - (515) 283-4209, +1 (515) 244-7694, +1 (515) 255-7047, +15157836401' etc. To maintain consistency, phone numbers were updated so that each phone is displyaed in national US format defined by python library "phonenumbers" which is in the format - (515) 283-4209. The code to audit the telephone numbers is as follows - 

In [58]:
## The following code can be used to audit telephone numbers and improve them
## The code can be found in audit_update_telephone_numbers.ipynb
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

#OSMFILE = 'example.osm'
OSMFILE = 'des_moines.osm'
#telephone_re = re.compile(r'^(\d{3})-(\d{3})-(\d{4})$') 
telephone_re = re.compile(r'^\(\d{3}\)\s(\d{3})-(\d{4})$') 

def audit_phone_number(osmfile):
    osm_file = open(osmfile, "r", encoding="utf8")
    phone_numbers = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)): # TODO: Learn about what events param does
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_number(tag):
                    m = telephone_re.search(tag.attrib['v'])
                    if m:
                        phone_numbers['correct'].add(m.group())
                    else:
                        phone_numbers['incorrect'].add(tag.attrib['v'])
    osm_file.close()
    return phone_numbers
    

def is_phone_number(elem):
    return (elem.attrib['k'] == "phone" or elem.attrib['k'] == "contact:phone")


The function is_phone_number is used to extract phone numbers from tags. 

The audit_phone_number function opens up the des moines Openstreet map file to be parsed. Then the function iterates through the file. If there is a node or way tag, then the "tag" tags in them are examined to see if they have telephone number. Then the function searches through the telephone_re regular expression to see if it finds a match for telephone number with US National format. If there is a match, it adds it in a dictionary of phone numbers for key "correct" and if the telephone number is present which have format other than US national format then it adds it in the dictionary of phone numbers for key "incorrect". 

The list of the audited telephone numbers is as follows - 

In [63]:
#def update phone_number()
phone_types = audit_phone_number(OSMFILE)
pprint.pprint(dict(phone_types))

{'correct': {'(515) 216-2760',
             '(515) 216-2762',
             '(515) 221-2751',
             '(515) 223-4597',
             '(515) 223-6113',
             '(515) 223-7700',
             '(515) 223-8151',
             '(515) 223-8506',
             '(515) 225-1193',
             '(515) 225-9330',
             '(515) 226-1729',
             '(515) 226-8753',
             '(515) 226-8921',
             '(515) 237-1386',
             '(515) 243-1286',
             '(515) 243-2747',
             '(515) 243-7152',
             '(515) 244-0790',
             '(515) 244-2111',
             '(515) 244-7718',
             '(515) 244-8813',
             '(515) 246-1390',
             '(515) 248-6315',
             '(515) 248-6316',
             '(515) 255-0007',
             '(515) 255-6213',
             '(515) 255-6720',
             '(515) 255-9231',
             '(515) 262-0640',
             '(515) 262-2108',
             '(515) 262-3069',
             '(515) 262-7956',
        

Once the auditing is complete, the function update_telephone_number updates all the phone numbers to match US National format. The function update_phone_number is as follows - 

In [61]:
import phonenumbers
def update_phone_number(phone_no):
    for match in phonenumbers.PhoneNumberMatcher(phone_no, "US"):
        return phonenumbers.format_number(match.number, phonenumbers.PhoneNumberFormat.NATIONAL)
OSMFILE = 'des_moines.osm'       
st_types = audit_phone_number(OSMFILE)


The update_phone_number function, with the help of regular expression searches for the US phone numbers in the tag and matches the format to update the phone numbers in standard US format. The output of the update_phone_number function is as follows -

In [64]:
for st_type, ways in st_types.items(): # changed this method from .iteritems() from 2.7 to 3.6's .items()  
        for name in ways:
            better_name = update_phone_number(name)
            print (name, "=>", better_name)  

+1 515 282 9948 => (515) 282-9948
515-274-9296 => (515) 274-9296
+1 515 279 1002 => (515) 279-1002
+15152821031 => (515) 282-1031
+1 515 262 6007 => (515) 262-6007
+1-515-225-6232 => (515) 225-6232
+15159635040 => (515) 963-5040
+15152774405 => (515) 277-4405
+1 515 277 0262 => (515) 277-0262
515-289-1703 => (515) 289-1703
+1 515 490 9647 => (515) 490-9647
+1 515 274 2211 => (515) 274-2211
+1 515 421 4404 => (515) 421-4404
+1 515 277 4629 => (515) 277-4629
+1 515 282 2334 => (515) 282-2334
+1 515 225 0410 => (515) 225-0410
+1 515-967-0773 => (515) 967-0773
+1 515 358 8000 => (515) 358-8000
+1 515 279 5947 => (515) 279-5947
515-255-2181 => (515) 255-2181
+1 515 559 3650 => (515) 559-3650
+1 515 953 0226 => (515) 953-0226
+1 515 274 5151 => (515) 274-5151
+1 515 244 0655 => (515) 244-0655
+1 515 528 2329 => (515) 528-2329
+15152443182 => (515) 244-3182
+1 515 279 4711 => (515) 279-4711
+1-515-986-1783 => (515) 986-1783
+1 515 247 3150 => (515) 247-3150
+1 515 242 7272 => (515) 242-7272
+

(515) 283-4213 => (515) 283-4213
(515) 279-9008 => (515) 279-9008
(515) 288-1050 => (515) 288-1050
(515) 282-3634 => (515) 282-3634
(515) 244-7718 => (515) 244-7718
(515) 223-4597 => (515) 223-4597
(515) 964-0656 => (515) 964-0656
(515) 216-2760 => (515) 216-2760
(515) 283-4046 => (515) 283-4046
(515) 255-0007 => (515) 255-0007
(515) 283-4060 => (515) 283-4060
(515) 285-6394 => (515) 285-6394
(515) 225-9330 => (515) 225-9330
(515) 271-1700 => (515) 271-1700
(515) 283-4500 => (515) 283-4500
(515) 282-9575 => (515) 282-9575
(515) 457-3176 => (515) 457-3176
(515) 284-1401 => (515) 284-1401
(515) 967-7676 => (515) 967-7676
(515) 283-1386 => (515) 283-1386
(515) 285-7133 => (515) 285-7133
(515) 288-0206 => (515) 288-0206
(515) 283-4950 => (515) 283-4950
(515) 287-1022 => (515) 287-1022
(515) 221-2751 => (515) 221-2751
(515) 528-2293 => (515) 528-2293
(515) 248-6316 => (515) 248-6316
(515) 274-1208 => (515) 274-1208
(515) 875-7000 => (515) 875-7000
(515) 964-0900 => (515) 964-0900
(515) 283-

<hr style="height:1px;color:black">

<a id='database'></a>
## Preparing for SQL Database 

After the data in the Des Moines Openstreet map file is audited and cleaned, the data is prepared to be inserted into a SQL database. The code to do that is as follows - 

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
### The code is found in prepare_for_database.py
import re


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Rd.,": "Road",
            "Ave": "Avenue",
            "ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Cir": "Circle",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Hwy":"Highway",
            "Ct": "Court",
            "Pkwy": "Parkway",
            "Ln": "Lane",
            "Sq": "Square",
            "Ste": "Suite",
            "Real":"Real",
            "Saratoga":"Saratoga",
            "Alameda": "Alameda",
            "Way": "Way",
            "Expressway": "Expressway"
            }

def update_name(name, mapping):
    
    m = street_type_re.search(name)
    new_name = name
    if m:
        street_type = m.group()
        if street_type in mapping:
            new_name = street_type_re.sub(mapping[street_type], name)  
    return new_name

import phonenumbers
def update_phone_number(phone_no):
    for match in phonenumbers.PhoneNumberMatcher(phone_no, "US"):
        return phonenumbers.format_number(match.number, phonenumbers.PhoneNumberFormat.NATIONAL)

import zipcodes
def update_zipcode(zipcode):
    #return re.sub('(\d{5})-\d{4}', '\\1', zipcode)
    #print(zipcode)
    b = re.findall(r'\d+', zipcode)
    if b: 
        return b[0]

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

#OSM_PATH = "san-jose_california_cleaned.osm"
OSM_PATH = "des_moines.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    #The "node" field should hold a dictionary of the following top level node attributes:
    if element.tag == 'node':
        for attributes in node_attr_fields :
            if attributes in element.attrib :
                node_attribs[attributes] = element.attrib[attributes]

        for elem in element:
            if elem.tag == 'tag':
                temp_tags = {}
                for tagname in NODE_TAGS_FIELDS:
                    if tagname == 'id':
                        temp_tags['id'] = element.attrib['id']
                    elif tagname == 'key':
                        key = elem.attrib['k']
                        m = PROBLEMCHARS.search(key)
                        if m:
                            continue
                        else:
                            if ':' in key:
                                temp_tags['type'], temp_tags['key'] = key.split(':', 1)
                                if key == "addr:street" :
                                    temp_tags['value'] = update_name(elem.attrib['v'],mapping)
                                elif key == "addr:postcode" :
                                    temp_tags['value'] = update_zipcode(elem.attrib['v'])
                                elif key == "contact:phone" :
                                    temp_tags['value'] = update_phone_number(elem.attrib['v'])
                                else :
                                    temp_tags['value'] = elem.attrib['v']
                            else:
                                if key == "phone":
                                    temp_tags['type'] = default_tag_type
                                    temp_tags['key'] = key
                                    temp_tags['value'] = update_phone_number(elem.attrib['v'])
                                else:
                                    temp_tags['type'] = default_tag_type
                                    temp_tags['key'] = key
                                    temp_tags['value'] = elem.attrib['v']

                tags.append(temp_tags)
    
    elif element.tag == 'way':
        for attributes in way_attr_fields :
            if attributes in element.attrib:
                way_attribs[attributes] = element.attrib[attributes]
            
        for counter, elem in enumerate(element):
            if elem.tag == 'nd':
                temp_way_nodes = {}
                for tagname in WAY_NODES_FIELDS:
                    if tagname == 'id':
                        temp_way_nodes['id'] = element.attrib['id']
                    elif tagname == 'node_id':
                        temp_way_nodes['node_id'] = elem.attrib['ref']
                    elif tagname == 'position':
                        temp_way_nodes['position'] = counter
                way_nodes.append(temp_way_nodes)

            if elem.tag == 'tag':
                temp_way_tags = {}
                for tagname in WAY_TAGS_FIELDS:
                    if tagname == 'id':
                        temp_way_tags['id'] = element.attrib['id']
                    elif tagname == 'key':
                        key = elem.attrib['k']
                        m = PROBLEMCHARS.search(key)
                        if m:
                            continue
                        else:
                            if ':' in key:
                                temp_way_tags['type'], temp_way_tags['key'] = key.split(':', 1)
                                if key == "addr:street" :
                                    temp_way_tags['value'] = update_name(elem.attrib['v'],mapping)
                                elif key == "addr:postcode" :
                                    temp_way_tags['value'] = update_zipcode(elem.attrib['v'])
                                elif key == "contact:phone" :
                                     temp_way_tags['value'] = update_phone_number(elem.attrib['v'])
                                else :
                                    temp_way_tags['value'] = elem.attrib['v']
                            else:
                                if key == "phone":
                                    temp_way_tags['type'] = default_tag_type
                                    temp_way_tags['key'] = key
                                    temp_way_tags['value'] = update_phone_number(elem.attrib['v'])
                                else:
                                    temp_way_tags['type'] = default_tag_type
                                    temp_way_tags['key'] = key
                                    temp_way_tags['value'] = elem.attrib['v']

                                
                tags.append(temp_way_tags)

    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

    
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)



To prepare the data to be inserted in the database, the elements in the OSM XML files were parsed. To write them to csv file, the elements were transformed from document to tabular format and then the csv files were imported into the database.

The main function here is process_map. First, it opens writeable files for the schema (the database schema) as per the defination of schema from schema.py. Then it uses function UnicodeDictWriter to write multiple rows to the csv files. process_map function lets the write object know that there are headers. It then iterates through each element which is of concern and with the help of get_element the element is filtered. Basically the function get_element, if it has got right tag gets all the elements fitting the condition and reports the next one when needed. After that, the function shape_element takes as input an interparse Element object and returns a dictionary which are used to write to csv files. These dictionaries are validated using validate_element function which validates that an element that passed conforms to the defined schema in schema.

In order to make sure that the elements are cleaned, update_name(), update_zipcode() and update_phone_number() functions are called in shape_element function().

The output consists of 5 csv files - nodes, nodes_tags, ways, ways_nodes, ways_tags. 

<hr style="height:1px;color:black">

<a id='eda'></a>
## Creating Database and Data Exploration

### Creating Data :

After all the csv files were created loaded with data, the next task was importing them in sqlite db directly via the terminal. 

To import the csv files, first a database named "desMoines" was created. After that, all the tables corresponding to the csv file were created and then all the csv files were imported into their respective tables. Following is the syntax for creating desMoines database - 

C:\>cd sqlite_windows

C:\sqlite_windows>sqlite3 desMoines.db
SQLite version 3.9.2 2015-11-02 18:31:45
Enter ".help" for usage hints.

Following is the syntax to create the tables - 

```

sqlite> CREATE TABLE nodes (
   ...>     id INTEGER PRIMARY KEY NOT NULL,
   ...>     lat REAL,
   ...>     lon REAL,
   ...>     user TEXT,
   ...>     uid INTEGER,
   ...>     version INTEGER,
   ...>     changeset INTEGER,
   ...>     timestamp TEXT
   ...> );
sqlite>
sqlite> CREATE TABLE nodes_tags (
   ...>     id INTEGER,
   ...>     key TEXT,
   ...>     value TEXT,
   ...>     type TEXT,
   ...>     FOREIGN KEY (id) REFERENCES nodes(id)
   ...> );
sqlite>
sqlite> CREATE TABLE ways (
   ...>     id INTEGER PRIMARY KEY NOT NULL,
   ...>     user TEXT,
   ...>     uid INTEGER,
   ...>     version TEXT,
   ...>     changeset INTEGER,
   ...>     timestamp TEXT
   ...> );
sqlite>
sqlite> CREATE TABLE ways_tags (
   ...>     id INTEGER NOT NULL,
   ...>     key TEXT NOT NULL,
   ...>     value TEXT NOT NULL,
   ...>     type TEXT,
   ...>     FOREIGN KEY (id) REFERENCES ways(id)
   ...> );
sqlite>
sqlite> CREATE TABLE ways_nodes (
   ...>     id INTEGER NOT NULL,
   ...>     node_id INTEGER NOT NULL,
   ...>     position INTEGER NOT NULL,
   ...>     FOREIGN KEY (id) REFERENCES ways(id),
   ...>     FOREIGN KEY (node_id) REFERENCES nodes(id)
   ...> );
```
The syntax to import the csv files is as follows - 
```
sqlite> .mode csv
sqlite> .import ways_nodes.csv ways_nodes
sqlite> .import ways_tags.csv ways_tags
sqlite> .import nodes.csv nodes
sqlite> .import nodes_tags.csv nodes_tags
sqlite> .import ways.csv ways
```

Once the data is imported into the tables, the next step was data exploration. 

### Data Exploration

### Data Overview 

The following section contains the basic statistics about the dataset, the sql queries used to gather them, and some additional ideas about the data in context.

#### File Sizes 
```
des_moines.osm ......... 187 MB
charlotte.db .......... 105 MB
nodes.csv ............. 79.9 MB
nodes_tags.csv ........ 1.29 MB
ways.csv .............. 3.79 MB
ways_tags.csv ......... 13.2 MB
ways_nodes.cv ......... 25.1 MB  
```

#### Number of Nodes
```
sqlite> select count(*) from nodes;
```
915603

#### Number of ways
```
sqlite> SELECT COUNT(*) FROM ways;
```
64668

From the above results, we can see that the number of nodes and number of ways match with our results got from python code. 

#### Number of unique users 
```
sqlite> SELECT COUNT(DISTINCT(e.uid))
   ...> FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;
```
457

#### Top 10 contributing users 
```
sqlite> SELECT e.user, COUNT(*) as num
   ...> FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
   ...> GROUP BY e.user
   ...> ORDER BY num DESC
   ...> LIMIT 10;

Jeff Ollies Bot|287253
Jeff Ollie|254153
iowahwyman|181992
woodpeck_fixbot|78494
jgruca|63775
BAC|16031
keithly12|11981
ChartingFool|6730
Brutus|5416
Milo|5343
```

#### Number of users appearing only once
```
sqlite> SELECT COUNT(*)
   ...> FROM
   ...>     (SELECT e.user, COUNT(*) as num
   ...>      FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
   ...>      GROUP BY e.user
   ...>      HAVING num=1)  u;
```
84



### Additional data Exploration

First we look at zipcodes in Des Moines city. 

#### Top 10 postal codes 
```
sqlite> SELECT tags.value, COUNT(*) as count
   ...> FROM (SELECT * FROM nodes_tags
   ...>   UNION ALL
   ...>       SELECT * FROM ways_tags) tags
   ...> WHERE tags.key like '%postcode%'
   ...> GROUP BY tags.value
   ...> ORDER BY count DESC limit 10;
'50266'|997
'50265'|825
'50322'|817
'50312'|757
'50325'|592
'50310'|490
'50309'|227
'50311'|226
'50021'|162
'50131'|159
```

The zipcodes 50312, 50310, 50309, 50311 belong to the Des Moines City, therefore we have 1700 zipcodes belonging to Des Moines City. The zipcodes 50266 and 50255 belong to West Des Moines city, a suburb in Des Moines Metropolitan area (total of 1822 zipcodes). The zipcode 50322 belongs to Urbandale, another suburb in Des Moines Metropolitan area, the zipcode 50325 belongs to Clive, the zipcode 50021 belongs to Ankeny, the zipcode 50131 belongs to Johnston. The cities described here are all suburbs or part of the Des Moines Metropolitan area. We can say, that the metro extract does not just include Des Moines city but also the surrounding cities in the sprawl. Another aggregation is performed to confirm this which is as follows - 

#### Sort Cities By Count, Descending

```
sqlite> SELECT tags.value, COUNT(*) as count
   ...> FROM (SELECT * FROM nodes_tags UNION ALL
   ...>       SELECT * FROM ways_tags) tags
   ...> WHERE tags.key LIKE '%city%'
   ...> GROUP BY tags.value
   ...> ORDER BY count DESC LIMIT 10;
b'West Des Moines'|1311
b'Des Moines'|1101
b'Ankeny'|274
b'Clive'|201
b'Johnston'|156
b'Urbandale'|144
b'Windsor Heights'|34
b'Waukee'|21
b'van meter'|15
b'Altoona'|10
```

The results from the above sql query confirms that the metro extract is Des Moines Metropolitan area and not just Des Moines City. 

Next a postcode is examined which is in Altoona city, a suburb. 

#### Find postcodes from Altoona city with zipcode 50009

```
sqlite> SELECT *
   ...> FROM nodes
   ...> WHERE id IN (SELECT DISTINCT(id) FROM nodes_tags WHERE key like '%postcode%' AND value like '%50009%');
354403072|41.6433048|-93.4963368|maxerickson|360392|3|46843414|2017-03-14T14:59:54Z
367080467|41.6430727|-93.4810078|maxerickson|360392|3|46843414|2017-03-14T14:59:54Z
2810066946|41.6398606|-93.4755425|1121 Venbury|2043542|1|21873757|2014-04-22T21:37:32Z
3299645061|41.6493224|-93.5090908|Nate Gould|2082834|1|28267618|2015-01-20T02:57:29Z
4734386600|41.6498526|-93.4730143|maxerickson|360392|1|46843414|2017-03-14T14:59:50Z
5145326136|41.6424295|-93.465783|Iowa Kid|703517|3|52612883|2017-10-04T03:21:54Z

```

Then, more information on postal id '354403072' is found using following query. 

#### More Information on node from Postal Id 
```
sqlite> SELECT *
   ...> FROM nodes_tags
   ...> WHERE id='354403072';
354403072|'city'|'Altoona'|'addr'
354403072|'housenumber'|'2720'|'addr'
354403072|'postcode'|'50009'|'addr'
354403072|'street'|'8th Street Southwest'|'addr'
354403072|'amenity'|'doctors'|'regular'
354403072|'name'|'UnityPoint Clinic Family Medicine \xe2\x80\x93 Altoona'|'regular'
354403072|'operator'|'UnityPoint Health'|'regular'
354403072|'phone'|'(515) 967-0133'|'regular'
354403072|'website'|'https://www.unitypoint.org/desmoines/clinic.aspx?id=104'|'regular'
sqlite>
```
From the above node information, we can see that the postal code belongs to the UnityPoint Health Clinic. 

Next, further data exploration is done on to gain insights from the city.  

#### Top 10 appearing amenities

```
sqlite> SELECT value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> WHERE key like '%amenity%'
   ...> GROUP BY value
   ...> ORDER BY num DESC
   ...> LIMIT 10;
 
'place_of_worship'|273
'restaurant'|191
'bench'|188
'fast_food'|132
'fountain'|83
'school'|64
'bar'|44
'cafe'|42
'grave_yard'|41
'fuel'|37
```

#### Biggest Religion

```
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags
   ...> JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value like '%place_of_worship%') i
   ...> ON nodes_tags.id=i.id
   ...> WHERE nodes_tags.key like '%religion%'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC
   ...> LIMIT 1;

'christian'|261
```

#### Most Popular cuisines 
```
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags
   ...>     JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value like '%restaurant%') i
   ...>     ON nodes_tags.id=i.id
   ...> WHERE nodes_tags.key like '%cuisine%'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC;
'american'|28
'pizza'|23
'mexican'|17
'chinese'|16
'sandwich'|11
'asian'|5
'steak_house'|4
'sushi'|4
'barbecue'|3
'indian'|3
'international'|3
'italian'|3
'japanese'|3
'seafood'|3
'chicken'|2
'french'|2
'mediterranean'|2
'regional'|2
'Chinese_Fast_Food'|1
'burger'|1
'deli'|1
'ecuadorian'|1
'greek'|1
'ice_cream'|1
'kebab'|1
'tacos;mexican'|1
'thai'|1
'thai;lao'|1
```

For Des Moines, a city in midwest of the United States, the results from sql queries like Biggest religion and Most Popular cuisines are not surprising. They are as expected. However, for cuisines we see that pizza and italian are listed as separate cuisines, similarly chinese and chinese_fast_food then mediterranean, greek and kebab are listed as separate cuisines. As a future work they can be combined. The next thing which is explored is number of cafes. 

#### Number of cafes
```
sqlite> SELECT nodes_tags.value, COUNT(*) as num
   ...> FROM nodes_tags
   ...>     JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value like '%coffee_shop%') i
   ...>     ON nodes_tags.id=i.id
   ...> WHERE nodes_tags.key like '%name%'
   ...> GROUP BY nodes_tags.value
   ...> ORDER BY num DESC;
'Starbucks'|10
'Caribou Coffee'|3
'Dunkin' Donuts'|2
'Scooter's'|2
'Grounds For Celebration'|2
'Friedrich's World Coffees'|1
'Building 3000'|1
'Caribou Coffee & Einstein Bros. Bagels'|1
'Corazon Coffee Roasters'|1
'Greenway Crossing'|1
'Inspired Grounds Cafe'|1
'Jordan Creek Town Center'|1
'La Barista Coffee'|1
'Mars Cafe'|1
'Normandy Plaza'|1
'Ritual Cafe'|1
'St. Kilda'|1
'Timbuktu Coffee Bar'|1
'Valley West Mall'|1
'West Glen Town Center'|1
```

As expected, the large number of coffeeshops are starbucks. However, from the results it can be seen that Jordan Creek Town Center, Valley West Mall, West Glen Town Center are listed as cafes however they are actually malls with a coffee shop either a starbucks or Caribou Coffee inside them. This data can be improved in future work for OpenStreetMap Data.




<hr style="height:1px;color:black">

<a id='idea'></a>
## Additional Ideas

Many datagaps and flaws like cuisines not represented correctly, multiple formats of telephone numbers, incorrect names of cafes etc. can be seen in OpenStreetMap data as it is free editable map made by people all around the world. There are multiple error Detecting Tools like MapRoulette, Keep Right, Osmose to check the OSM data for potential data errors, inaccuracy or sparsely mapped place. Users should be made aware of these tools and should be encouraged to use these tools when they are adding new data or modifying existing data. MapRoulette uses the concept of a gamified approach to fixing OSM bugs that breaks common OpenStreetMap data problems into micro tasks. “Gamification” as a motivating force for contribution should be implemented more to encourage users to use one of the error detecting tools to make sure we get less error-prone data. There should be incentives like rewards, leaderships or badges for fixing certain number of bugs. 

Apart from the gamification approach, there should be either "automated" or "manual" setup to review and validate data on regular basis. Although this step seems simple, considering the sheer amount of data which is present in OpenStreetMapData, it would be very time consuming if the data has to be validated on daily basis, the ideal approach would be to validate once every two weeks or so.

One of the limitations which have been in OpenStreetMap Data is contributor's bias. Each contributor makes decisions about the types of features he or she will place on the map and the places he or she will map. As a result we have little idea of who created the map and why. A way to fix this is inclusion of usage of social checkins on the map. Even though city centers are well mapped, the completeness falls off quickly from the center and there’s still a lot of population and activity in the outskirts. Although, social network activity is not an absolute indicator of where general populations are or what areas important to map, it’s an useful heuristic which can be derived on a global level.

To implement all these approaches though, there would be a demand and constraint on resources like dedicated teams in different areas of expertise like engineering, management etc. There would also be need for proper softwares and budget as well.  

<hr style="height:1px;color:black">

<a id="conclusions"></a>
## Conclusions

The project on Wrangling OpenStreetMap Data helped understand why data cleaning is a crucial step in data analysis process. 
It also helped realize that if the data is not cleaned, then Information collection and application cannot be done. 
It helped understand the data munging techniques to assess the quality of the data for validity, accuracy, completeness, consistency and uniformity using Python. It also helped to learn how to store, query, and aggregate data using SQL.

#### Future Work 
Overall, the OpenstreetMap Data could be made cleaner and consistent for futher analysis. The future task would be cleaning and improving names of the cities, amenities etc to maintain consistency. With the clean data in hand, the data would be visualized using different plotting tools in python for further analysis. 
